In [1]:
!git clone https://github.com/zengjichuan/DANN.git

Cloning into 'DANN'...
remote: Enumerating objects: 26, done.
remote: Total 26 (delta 0), reused 0 (delta 0), pack-reused 26
Unpacking objects: 100% (26/26), 1.76 MiB | 5.95 MiB/s, done.


In [2]:
cd DANN

/kaggle/working/DANN


In [15]:
import random
import os
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from torchvision import datasets
from torchvision import transforms
import numpy as np
from torch.autograd import Function
import torch.nn as nn

In [4]:
from dataset.data_loader import *
from models.img_cnn_model import *
from train.test import *

ModuleNotFoundError: No module named 'functions'

In [5]:
import torch.utils.data as data
import os
import json
import gensim

In [35]:
class MnistDataloader(object):
    def __init__(self, training_images_filepath,training_labels_filepath,
                 test_images_filepath, test_labels_filepath):
        self.training_images_filepath = training_images_filepath
        self.training_labels_filepath = training_labels_filepath
        self.test_images_filepath = test_images_filepath
        self.test_labels_filepath = test_labels_filepath
    
    def read_images_labels(self, images_filepath, labels_filepath):        
        labels = []
        with open(labels_filepath, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049, got {}'.format(magic))
            labels = array("B", file.read())        
        
        with open(images_filepath, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051, got {}'.format(magic))
            image_data = array("B", file.read())        
        images = []
        for i in range(size):
            images.append([0] * rows * cols)
        for i in range(size):
            img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
            img = img.reshape(28, 28)
            images[i][:] = img            
        
        return images, labels
            
    def load_data(self):
        x_train, y_train = self.read_images_labels(self.training_images_filepath, self.training_labels_filepath)
        x_test, y_test = self.read_images_labels(self.test_images_filepath, self.test_labels_filepath)
        return (x_train, y_train),(x_test, y_test)   

In [39]:
training_images_filepath = '/kaggle/input/domain-adaptation-with-3d-printer-images/mnist/train-images-idx3-ubyte/train-images-idx3-ubyte'
training_labels_filepath = '/kaggle/input/domain-adaptation-with-3d-printer-images/mnist/train-labels-idx1-ubyte/train-labels-idx1-ubyte'
test_images_filepath = '/kaggle/input/domain-adaptation-with-3d-printer-images/mnist/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte'
test_labels_filepath = '/kaggle/input/domain-adaptation-with-3d-printer-images/mnist/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte'

In [45]:
import numpy as np # linear algebra
import struct
from array import array
from os.path  import join
from matplotlib import pyplot as plt

In [42]:
mnist_dataloader = MnistDataloader(training_images_filepath, training_labels_filepath, test_images_filepath, test_labels_filepath)
(x_train, y_train), (x_test, y_test) = mnist_dataloader.load_data()

In [61]:
!pwd

/kaggle/working/DANN


In [111]:
import numpy as np
from torchvision.datasets import MNIST, SVHN
from torchvision import transforms
import torch
from torch.utils.data import DataLoader


def normalize(data_tensor):
    '''re-scale image values to [-1, 1]'''
    return (data_tensor / 255.) * 2. - 1. 

def tile_image(image):
    '''duplicate along channel axis'''
    return image.repeat(3,1,1)

transform_list = [transforms.ToTensor(), transforms.Lambda(lambda x: normalize(x))]

batch_size = 64

mnist_dataset = MNIST(root="data", train=True, download=True,
    transform=transforms.Compose(transform_list+[
        transforms.ToPILImage(), 
        transforms.Resize(32), # resize to [32,32]
        transforms.ToTensor(),
        transforms.Lambda(lambda x: tile_image(x))
        ]))
# raw mnist images: [num, 28, 28] range (0, 255)

mnist_test_dataset = MNIST(root="data", train=False, download=True,
    transform=transforms.Compose(transform_list+[
        transforms.ToPILImage(), 
        transforms.Resize(32), # resize to [32,32]
        transforms.ToTensor(),
        transforms.Lambda(lambda x: tile_image(x))
        ]))

svhn_dataset = SVHN(root="data", split='train', download=True,
    transform=transforms.Compose(transform_list)) # [num, 3, 32, 32] range (0, 255)


svhn_test_dataset = SVHN(root="data", split='test', download=True,
    transform=transforms.Compose(transform_list))


# iterate with data loader
svhn_loader = DataLoader(dataset=svhn_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0)
for batch_idx, (data, target) in enumerate(svhn_loader):
    print(data.size())
    break
    
svhn_test_loader = DataLoader(dataset=svhn_test_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0)
for batch_idx, (data, target) in enumerate(svhn_loader):
    print(data.size())
    break

mnist_loader = DataLoader(dataset=mnist_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0)
for batch_idx, (data, target) in enumerate(mnist_loader):
    print(data.size())
    break

mnist_test_loader = DataLoader(dataset=mnist_test_dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0)
for batch_idx, (data, target) in enumerate(mnist_loader):
    print(data.size())
    break

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])
torch.Size([64, 3, 32, 32])


In [112]:
cuda = False
cudnn.benchmark = True
lr = 1e-3
batch_size = 64
n_epoch = 100
image_size = 32

manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# dataset_source_train = GetLoader(data_fn=source_train,)
# dataset_source_eval = GetLoader(data_fn=source_eval,)
# dataset_target_train = GetLoader(data_fn=target_train,)

dataloader_source_train = mnist_loader #torch.utils.data.DataLoader(dataset=dataset_source_train, batch_size=batch_size, shuffle=True)
dataloader_source_test = mnist_test_loader #torch.utils.data.DataLoader(dataset=dataset_source_eval, batch_size=batch_size, shuffle=False)
dataloader_target_train = svhn_loader #torch.utils.data.DataLoader(dataset=dataset_target_train, batch_size=batch_size, shuffle=True)
dataloader_target_test = svhn_test_loader #torch.utils.data.DataLoader(dataset=dataset_target_train, batch_size=batch_size, shuffle=True)


In [13]:
class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [105]:
class CNNModel(nn.Module):

    def __init__(self):
        super(CNNModel, self).__init__()
        self.LeNet = nn.Sequential()
        self.LeNet.add_module('f_conv1', nn.Conv2d(3, 64, kernel_size=5))
        self.LeNet.add_module('f_bn1', nn.BatchNorm2d(64))
        self.LeNet.add_module('f_pool1', nn.MaxPool2d(2))
        self.LeNet.add_module('f_relu1', nn.ReLU(True))
        self.LeNet.add_module('f_conv2', nn.Conv2d(64, 50, kernel_size=5))
        self.LeNet.add_module('f_bn2', nn.BatchNorm2d(50))
        self.LeNet.add_module('f_drop1', nn.Dropout2d())
        self.LeNet.add_module('f_pool2', nn.MaxPool2d(2))
        self.LeNet.add_module('f_relu2', nn.ReLU(True))

        self.class_classifier = nn.Sequential()
        self.class_classifier.add_module('c_fc1', nn.Linear(50 * 5 * 5, 100))
        self.class_classifier.add_module('c_bn1', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu1', nn.ReLU(True))
        self.class_classifier.add_module('c_drop1', nn.Dropout())
        self.class_classifier.add_module('c_fc2', nn.Linear(100, 100))
        self.class_classifier.add_module('c_bn2', nn.BatchNorm1d(100))
        self.class_classifier.add_module('c_relu2', nn.ReLU(True))
        self.class_classifier.add_module('c_fc3', nn.Linear(100, 10))
        self.class_classifier.add_module('c_softmax', nn.LogSoftmax())

        self.domain_classifier = nn.Sequential()
        self.domain_classifier.add_module('d_fc1', nn.Linear(50 * 5 * 5, 100))
        self.domain_classifier.add_module('d_bn1', nn.BatchNorm1d(100))
        self.domain_classifier.add_module('d_relu1', nn.ReLU(True))
        self.domain_classifier.add_module('d_fc2', nn.Linear(100, 2))
        self.domain_classifier.add_module('d_softmax', nn.LogSoftmax())

    def forward(self, input_data, alpha):
        input_data = input_data.expand(input_data.data.shape[0], 3, 32, 32)
        feature = self.LeNet(input_data)
        feature = feature.view(-1, 50 * 5 * 5)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.class_classifier(feature)
        domain_output = self.domain_classifier(reverse_feature)

        return class_output, domain_output

my_net = CNNModel()

In [83]:
optimizer = optim.Adam(my_net.parameters(), lr=lr)

loss_class = torch.nn.NLLLoss()
loss_domain = torch.nn.NLLLoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_domain = loss_domain.cuda()

for p in my_net.parameters():
    p.requires_grad = True

In [ ]:
print_count = 5
for epoch in range(n_epoch):

    len_dataloader = min(len(dataloader_source_train), len(dataloader_target_train))
    data_source_iter = iter(dataloader_source_train)
    data_target_iter = iter(dataloader_target_train)

    i = 0
    while i < len_dataloader:

        p = float(i + epoch * len_dataloader) / n_epoch / len_dataloader
        alpha = 2. / (1. + np.exp(-10 * p)) - 1

        # training model using source data
        data_source = next(data_source_iter)
        s_img, s_label = data_source

        my_net.zero_grad()
        batch_size = len(s_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)
        domain_label = torch.zeros(batch_size)
        domain_label = domain_label.long()

        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)
        inputv_img = Variable(input_img)
        classv_label = Variable(class_label)
        domainv_label = Variable(domain_label)

        class_output, domain_output = my_net(input_data=inputv_img, alpha=alpha)
        err_s_label = loss_class(class_output, classv_label)
        err_s_domain = loss_domain(domain_output, domainv_label)

        # training model using target data
        data_target = next(data_target_iter)
        t_img, _ = data_target

        batch_size = len(t_img)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        domain_label = torch.ones(batch_size)
        domain_label = domain_label.long()

        if cuda:
            t_img = t_img.cuda()
            input_img = input_img.cuda()
            domain_label = domain_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        inputv_img = Variable(input_img)
        domainv_label = Variable(domain_label)

        _, domain_output = my_net(input_data=inputv_img, alpha=alpha)
        err_t_domain = loss_domain(domain_output, domainv_label)
        err = err_t_domain + err_s_domain + err_s_label
        err.backward()
        optimizer.step()

        i += 1
        if i % (1 + len_dataloader // print_count) == 0:
            print ('Epoch: {} [iter: {} / {}] -> err_s_label: {:.3f}, err_s_domain: {:.3f}, err_t_domain: {:.3f}'
               .format(epoch, i, len_dataloader, err_s_label.cpu().data.numpy(),
                 err_s_domain.cpu().data.numpy(), err_t_domain.cpu().data.numpy()))

    test(mnist_test_loader, my_net, epoch)
    test(svhn_test_loader, my_net, epoch)

Epoch: 0 [iter: 188 / 937] -> err_s_label: 2.301, err_s_domain: 0.717, err_t_domain: 0.669
Epoch: 0 [iter: 376 / 937] -> err_s_label: 2.299, err_s_domain: 0.717, err_t_domain: 0.669
Epoch: 0 [iter: 564 / 937] -> err_s_label: 2.299, err_s_domain: 0.717, err_t_domain: 0.669
Epoch: 0 [iter: 752 / 937] -> err_s_label: 2.316, err_s_domain: 0.717, err_t_domain: 0.669
Epoch: 0 accuracy: 0.103
Epoch: 0 accuracy: 0.159
Epoch: 1 [iter: 188 / 937] -> err_s_label: 2.331, err_s_domain: 0.717, err_t_domain: 0.669
Epoch: 1 [iter: 376 / 937] -> err_s_label: 2.312, err_s_domain: 0.717, err_t_domain: 0.669
Epoch: 1 [iter: 564 / 937] -> err_s_label: 2.306, err_s_domain: 0.717, err_t_domain: 0.669
Epoch: 1 [iter: 752 / 937] -> err_s_label: 2.296, err_s_domain: 0.717, err_t_domain: 0.669
Epoch: 1 accuracy: 0.103
Epoch: 1 accuracy: 0.159
Epoch: 2 [iter: 188 / 937] -> err_s_label: 2.301, err_s_domain: 0.717, err_t_domain: 0.669
Epoch: 2 [iter: 376 / 937] -> err_s_label: 2.289, err_s_domain: 0.717, err_t_doma

In [133]:
def test(dataloader, my_net, epoch):
    cuda = False
    cudnn.benchmark = True
    my_net = my_net.eval()

    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(dataloader)
    data_target_iter = iter(dataloader)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        # test model using target data
        data_target = next(data_target_iter)
        t_img, t_label = data_target

        batch_size = len(t_label)

        input_img = torch.FloatTensor(batch_size, 3, image_size, image_size)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            t_img = t_img.cuda()
            t_label = t_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        class_label.resize_as_(t_label).copy_(t_label)
        inputv_img = Variable(input_img)
        classv_label = Variable(class_label)

        class_output, _ = my_net(input_data=inputv_img, alpha=alpha)
        pred = class_output.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(classv_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct * 1.0 / n_total

    print ('Epoch: {} accuracy: {:.3f}'.format(epoch, accu))

In [ ]:
torch.save(my_net, 'mnist_svhn_model.pth'.format(model_root, epoch))